<a href="https://colab.research.google.com/github/DucKhiemPJ/AI-learning/blob/main/train_on_gei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch torchvision numpy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
!git clone https://github.com/itijyou/gaitrecog.git
%cd gaitrecog


Cloning into 'gaitrecog'...
remote: Enumerating objects: 78, done.
remote: Total 78 (delta 0), reused 0 (delta 0), pack-reused 78 (from 1)
Receiving objects: 100% (78/78), 19.46 KiB | 19.46 MiB/s, done.
Resolving deltas: 100% (26/26), done.
/content/gaitrecog


In [3]:
!pip install fastai==1.0.61


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.2/239.2 kB 18.0 MB/s eta 0:00:00
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19173 sha256=21f4419703198654761c2571a05a93cdd017db02c5b99f1cb7345983edb65514
  Stored in directory: /root/.cache/pip/wheels/47/50/9e/29dc79037d74c3c1bb4a8661fb608e8674b7e4260d6a3f8f51
Successfully built nvidia-ml-py3
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.19
    Uninstalling fastai-2.7.19:
      Successfully uninstalled fastai-2.7.19


In [4]:
%cd /content/gaitrecog
from train_on_gei import *

/content/gaitrecog


In [2]:
torch.cuda.set_device(0)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Train

In [5]:
!tar -xvzf /content/gaitrecog/casiab-nm.tar.gz -C /content/gaitrecog/


casiab-nm/
casiab-nm/labels.csv
casiab-nm/data.pkl


In [6]:
import random
import numpy as np
import torch

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # Nếu dùng GPU
    torch.backends.cudnn.deterministic = True  # Đảm bảo kết quả giống nhau
    torch.backends.cudnn.benchmark = False  # Giảm độ biến thiên
set_seed(0)


In [7]:
dataset = '/content/gaitrecog/casiab-nm'
# ends of (tr,vl,ts) splits
splitset = '50,74,124'
# train with the first 74 persons
task,split = 'tr','tv'
# LB model
model = 'lb'
# MT model
#model = 'mt'
# Siamese model
#model = 's'

opt,lr,mom,wd,bs,epochs,sched = 'sgd',0.01,0.9,0.0005,128,240,'st-14-96'

In [9]:
data,data_mean = get_data(dataset, splitset, bs, task, split)

In [10]:
get_net = {'lb':LBNet,'mt':MTNet,'s':SiameseNet,'d':DebugNet}.get(model, None)
if get_net: net = get_net(data_mean=data_mean)
else: assert False, 'Not implemented for model {}.'.format(model)
model_dir = Path('output')/dataset
assert opt=='sgd', f'Unknown opt method {opt}'
opt_func = partial(SGDEx, momentum=mom)
learn = LearnerEx(data, net, opt_func=opt_func, metrics=accuracy,
                  true_wd=False, wd=wd, path=Path('..'), model_dir=model_dir)
learn.callback_fns[0] = partial(RecorderEx, add_time=learn.add_time)

In [11]:
model_name = f'{dataset}_{model}_{opt}-{lr}-{mom}-{wd}_{sched}_bs{bs}_{split}'
assert sched.startswith('st')
iters = array([float(x)*10000 for x in sched.split('-')[1:]])
batches = len(data.train_dl) * epochs
assert np.all(iters < batches)
iters = np.append(iters, batches).astype(int)
phs = [TrainingPhase(x).schedule_hp('lr', lr*0.1**i) for i,x in enumerate(iters)]
learn.callback_fns += [
    partial(GeneralScheduler, phases=phs),
    partial(SaveModelCallback, every='epoch', name=model_name),
]

In [ ]:
learn.fit(epochs, 1)
learn.recorder.plot_losses()

epoch,train_loss,valid_loss,accuracy,recog_acc,time


/content/gaitrecog/train_on_gei.py:332: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha = 1) (Triggered internally at /pytorch/torch/csrc/utils/python_arg_parser.cpp:1661.)
  buf.add_(-group['lr'], p.grad)


Test

In [ ]:
data,_ = get_data(dataset, splitset, bs, 'ts', 'ts')

In [ ]:
learn = LearnerEx(data, net, metrics=accuracy, path=Path('..'), model_dir=model_dir)
learn.create_opt(lr, wd)
learn.callback_fns[0] = partial(RecorderEx, add_time=learn.add_time)
learn.callbacks += [learn.callback_fns[0](learn)]
learn.load('casiab-nm_lb_sgd-0.01-0.9-0.0005_st-14-96_bs128_tv_239', purge=False)
preds = learn.get_preds()

In [ ]:
xl = learn.data.valid_dl.x
# acc.shape is (probe,gallery)
acc = RecorderEx.calc_acc(learn.recorder.preds, xl.items1.inner_df, xl.items2.inner_df)
pacc = array([j[array(chain(range(i),range(i+1,acc.shape[1])))] for i,j in enumerate(acc)])
with np_print_options(formatter={'float':'{:1.7f}'.format}, threshold=sys.maxsize):
    print(pacc.mean())
    print(pacc.mean(1))
    print(acc)